<a href="https://colab.research.google.com/github/seiji0203/BERT/blob/master/%E7%AC%AC6%E7%AB%A0%EF%BC%BF%E6%96%87%E7%AB%A0%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 6-1
!mkdir chap6
%cd ./chap6

/content/chap6


In [ ]:
# 6-2
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.10

     |████████████████████████████████| 2.1 MB 13.6 MB/s 
     |████████████████████████████████| 486 kB 49.4 MB/s 
     |████████████████████████████████| 13.4 MB 36.4 MB/s 
     |████████████████████████████████| 841 kB 50.3 MB/s 
     |████████████████████████████████| 895 kB 53.8 MB/s 
     |████████████████████████████████| 3.3 MB 51.7 MB/s 
     |████████████████████████████████| 829 kB 43.3 MB/s 
     |████████████████████████████████| 176 kB 51.3 MB/s 
     |████████████████████████████████| 125 kB 76.9 MB/s 
     |████████████████████████████████| 596 kB 58.7 MB/s 
     |████████████████████████████████| 1.3 MB 58.0 MB/s 
     |████████████████████████████████| 271 kB 63.8 MB/s 
     |████████████████████████████████| 160 kB 59.9 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=e1e82852f714182e3cb88a6cbabcfd607238d5d64ae5b0a4ba185a231809cf30
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d78

In [ ]:
# 6-3
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [ ]:
# 文章分類（ネガポジ判定：感情分析）

tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
bert_sc = bert_sc.cuda()

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [ ]:
# 精度＝予測が正しかったデータの数/全体のデータ数
# 6-5
text_list = ["この映画は面白かった。", "この映画の最後にはがっかりさせられた。", "この映画を見て幸せな気持ちになった。"]
label_list = [1,0,1]

# データの符号化
encoding = tokenizer(text_list, padding = 'longest', return_tensors='pt')
encoding = { k: v.cuda() for k, v in encoding.items() }
labels = torch.tensor(label_list).cuda()

# 推論 (推論はtorch.no_gradの中で行う)
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits # 分類スコア
labels_predicted = scores.argmax(-1) # スコアが最も高いラベル
num_correct = (labels_predicted==labels).sum().item() # 正解数
accuracy = num_correct/labels.size(0) # 精度

print("# scoresのサイズ:")
print(scores.size())
print("# predicted labels:")
print(labels_predicted)
print("# accuracy:")
print(accuracy)

# scoresのサイズ:
torch.Size([3, 2])
# predicted labels:
tensor([1, 1, 1], device='cuda:0')
# accuracy:
0.6666666666666666


In [ ]:
# 6-6
# 符号化
encoding = tokenizer(text_list, padding='longest', return_tensors='pt')
encoding['labels'] = torch.tensor(label_list) # 入力にラベルを加える
encoding = {k: v.cuda() for k, v in encoding.items()}

# ロスの計算
output = bert_sc(**encoding)
loss = output.loss # 損失の取得
print(loss)

tensor(0.7277, device='cuda:0', grad_fn=<NllLossBackward>)


In [ ]:
# liveboor ニュースコーパスコーパス
# データのダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
# ファイルの解凍
!tar -zxf ldcc-20140209.tar.gz

--2021-10-19 05:14:43--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  3.15MB/s    in 2.7s    

2021-10-19 05:14:47 (3.15 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [ ]:
!cat ./text/it-life-hack/it-life-hack-6342280.txt # ファイルを表示(記事を解析するときは記事本文の4行目以降から解析する)

http://news.livedoor.com/article/detail/6342280/
2012-03-06T13:00:00+0900
USB3.0対応で爆速データ転送！　9倍速のリーダー／ライター登場
USB3.0が登場してから今年で4年目となるがパソコン側でのUSB3.0ポート搭載が進んで来ても対応機器がなかなか充実していない現状がある。そんな中で新しく高速な読み取りが可能なメモリーカードリーダー／ライターが登場した。

バッファローコクヨサプライがUSB3.0対応のカードリーダー／ライターを発表した。SDHC対応のSD系メディアやコンパクトフラッシュ、メモリースティック系メディア、xDピクチャーカードといったデジカメやスマホ、携帯ゲームといった機器で使われている各種メディアを従来よりも短時間でPCに取り込むことが可能になる。

転送速度が5Gbps（理論値）とUSB2.0の480Mbpsと比べて爆速になったUSB3.0はPC側の対応が進んで来ていたが高速転送が生かせる周辺機器としては、外付けHDDや一部のUSBメモリーくらいしかなかった。これに多くのメディアが扱えるリーダー／ライターが加わることで手軽にUSB3.0の恩恵を受けることができるようになる。

今回発表されたのは、USB3.0ケーブルとカードリーダー本体が分かれるタイプの「BSCR09U3」シリーズ（3,240円）、USB3.0コネクタをカードリーダー本体に内蔵している「BSCRD04U3」シリーズ（2,690円）だ。共にホワイトとブラックのカラーバリエーションが用意される（発売は3月下旬以降）。

■リリースページ
■バッファローコクヨサプライ




■バッファローの記事をもっと見る
・約283gでカバンに入る！小型キーボードの驚くべき機能
・3種類のホットキーで使いやすい！AndroidとPCで使えるキーボードの魅力
・ドラえもんもビックリの新アイテム！マウスとキーボードが合体"OPAir"
・ありそうでなかった便利機能！ファイル仕分けする画期的なHDD


サンディスク SanDisk microSDHC 32GB（microSD 32GB） 超高速クラス4  変換アダプター付 世界国内シェアNo.1 バルク品
クチコミを見る


In [ ]:
# BERTのファインチューニングと性能評価
# データローダ（データセットを前処理してBERTに入力可能な形式にする）
dataset_for_loader = [
                      {'data':torch.tensor([0,1]), 'labels':torch.tensor(0)},
                      {'data':torch.tensor([2,3]), 'labels':torch.tensor(1)},
                      {'data':torch.tensor([4,5]), 'labels':torch.tensor(2)},
                      {'data':torch.tensor([6,7]), 'labels':torch.tensor(3)}]
loader = DataLoader(dataset_for_loader, batch_size=2, shuffle=True)

# データセットからミニバッチを取り出す
for idx, batch in enumerate(loader):
  print(f'# batch {idx}')
  print(batch)
# ファインチューニングではここでバッチごとの処理を行う

# batch 0
{'data': tensor([[2, 3],
        [4, 5]]), 'labels': tensor([1, 2])}
# batch 1
{'data': tensor([[6, 7],
        [0, 1]]), 'labels': tensor([3, 0])}


In [ ]:
# データの前処理（学習・検証・テストデータに分類）
# 6-11 カテゴリのリスト
category_list = ['dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch', 'topic-news']
# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 各データの形式を整える
max_length = 128
dataset_for_loader = []
for label, category in enumerate(tqdm(category_list)):
    for file in glob.glob(f'./text/{category}/{category}*'):
        lines = open(file).read().splitlines()
        text = '\n'.join(lines[3:]) # ファイルの4行目からを抜き出す。
        encoding = tokenizer(
            text,
            max_length=max_length, 
            padding='max_length',
            truncation=True
        )
        encoding['labels'] = label # ラベルを追加
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }
        dataset_for_loader.append(encoding)

100%|██████████| 9/9 [00:30<00:00,  3.34s/it]


In [ ]:
print(dataset_for_loader[0])

{'input_ids': tensor([    2,   969,     5,  5386,   320,     6,  2198,    49,  6675,    13,
         5871,    12,  5023, 10776,   158,    14,  1218,   120,     8,   218,
            9,     6, 14163,    12,     9,    80,  3635,   205,    29,     8,
           36,   744,  8065,   185,   721,     7,   139,    82,    38,    36,
          185,   721,    11,  2495,    34,    82,    38,    36,    53,     7,
        18786,   832,    11,  1138,    80,    82,    38, 19066, 19066, 28448,
        14163,     7,  8034, 13606,     9,     6,    53,   230,    59, 21875,
            9,  1447,    13,     9,  4422,     6,   969,  2032, 14163,  1157,
           12, 10889,  1628, 28452, 28512,     9,     6,  1650,    11,  5197,
           34,   173,    13,  5517,     8,   218,  7046,     7, 24633,    49,
        15258,    14,  2824,    45,    28,    31,   124,    75,     8,  1011,
         1151,     5,   853, 29815,  2375,    23, 12166,    35,  2538,   573,
           24,     9,     6, 14163,  1157,    12, 

In [ ]:
# 6-13
# dataset の分割
random.shuffle(dataset_for_loader)# ランダムにシャッフル
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:]# テストデータ

# データセットからデータローダを作成
# 学習データは学習データはshuffle=Trueにする
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True) # 32 論文値
dataloader_val = DataLoader(dataset_val, batch_size=256) # 256 勾配の計算が必要ないため
dataloader_test = DataLoader(dataset_test, batch_size=256)

In [ ]:
# PyTorch Lightningnによるファインチューニングとテスト

# 6-14
class BertForSequenceClassification_pl(pl.LightningModule):
        
    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(model_name,
                                                                     num_labels=num_labels)
        
    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss
        
    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる。

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels') # バッチからラベルを取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) #精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [ ]:
# 6-15　ファインチューニングの設定
# 学習時にモデルの重みを保存する条件を指定（PyTorch Lightningのデフォルトでは1エポックが終わるごとに損失値が計算される）
checkpoint = pl.callbacks.ModelCheckpoint(monitor='val_loss', # val_lossを監視する
                                          mode='min',  # val_lossが小さいモデルを保存
                                          save_top_k=1,  # val_lossが最も小さいベストモデルのみを保存する
                                          save_weights_only=True,  #モデルの重みのみを保存する 
                                          dirpath='model/',) # モデルファイルを保存するディレクトリ

# 学習の方法を指定
trainer = pl.Trainer(gpus=1, # 学習に学習にGPUを一つ使う
                     max_epochs=10, #  学習をを10エポックごとに行う
                     callbacks = [checkpoint]) # どのような時にモデルの重みを保存するか指定

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
# 6-16 PyTorch Lightningモデルのダウンロード
model = BertForSequenceClassification_pl(MODEL_NAME, num_labels=9, lr=1e-5)

# ファインチューニングを行う
trainer.fit(model, dataloader_train, dataloader_val)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
best_model_path = checkpoint.best_model_path # file of bestmodel
print('ベストモデルのファイル:', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失:', checkpoint.best_model_score)

In [ ]:
# 6-18
%load_ext tensorboard
%tensorboard --logdir ./

In [ ]:
# 6-19 今回のモデルはバッチサイズズ32、学習率率1e-5で行なった。一般にはいくつかパターンをためして最も良かったものをしようするのが一般的である
test = trainer.test(test_dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

In [ ]:
# 6-20  PyTorch Lightningモデルのロードロード
model = BertForSequenceClassification_pl.load_from_checkpoint(best_model_path)

# Transformers対応のモデルをモデルを./model_trainformersに保存
model.bert_sc.save_pretrained('./model_transformers')

In [ ]:
# 6-21
bert_sc = BertForSequenceClassification.from_pretrained('./model_transformers')